# Relatório Hands-On-1

Fernando Antonio Marques Schettini $^1$, Gabriel Mascarenhas Costa de Sousa$^2$, Jadson Nobre das Virgens$^2$

$^1$ Curso de Engenharia de Computação - Centro universitário SENAI CIMATEC, Salvador, Bahia, Brazil  

$^2$ Curso de Sistemas de Informação - Universidade do Estado da Bahia, Salvador, Bahia, Brazil

# Resumo

Este é o relátorio das atividades realizadas durante a execução da prática Hands-On-1$^{[1]}$. O relatório foi feito como atividade avaliativa da matéria Fundamentos de Programação Paralela, lecionada no centro universitário SENAI CIMATEC. 

A prática Hands-On-1, dividida em 2 seções, tem o objetivo de introduzir conceitos de programação paralela através da aplicação das técnicas de paralelização utilizando a bilbioteca OPENMP em códigos em C para a otimização do tempo de execução.

# Introdução

Na sessão um, somos apresentados a uma situação problema: Multiplicação de Matrizes. Um código de multiplicação de matrizes é problemático em relação ao tempo de execução, já que o número de operações cresce de forma exponecial a medida em que o aumentamos o tamanho das matrizes, característico de um algoritmo de complexidade O(n$^2$). Por isso, nessa sessão utilizamos a biblioteca OPENMP para dividir os loops responsáveis pela multiplicação dos itens entre as threads do processador.

Fazendo um profilling do código, podemos localizar os pontos aonde o código demora mais. Para isso, primeiro compilamos o código, executamos, e utilizamos a ferramenta gprof para nos fornecer uma ánalise sobre a relação entre as partes do código e seus respectivos tempos de execução:

In [6]:
!gcc mm.c -o mm -fopenmp -pg

In [3]:
!./mm 1000

1000	1.621535


In [5]:
!gprof -b mm gmon.out

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  ms/call  ms/call  name    
 99.73      4.17     4.17                             main
  0.48      4.19     0.02        2    10.08    10.08  initializeMatrix

			Call graph


granularity: each sample hit covers 2 byte(s) for 0.24% of 4.19 seconds

index % time    self  children    called     name
                                                 <spontaneous>
[1]    100.0    4.17    0.02                 main [1]
                0.02    0.00       2/2           initializeMatrix [2]
-----------------------------------------------
                0.02    0.00       2/2           main [1]
[2]      0.5    0.02    0.00       2         initializeMatrix [2]
-----------------------------------------------

Index by function name

   [2] initializeMatrix        [1] main


Como o esperado, os três laços responsáveis pela operação da multiplicação, são a parte do código que levam mais tempo para serem executadas, por isso, aplicaremos as técnicas de paralelização. 

Na sessão 2, temos um código que basea-se na otimização de multiplicação de itens dentro de uma matriz por coeficientes específicos, repartindo o trabalho entre as threads numa mesma matriz e simulando a execução de uma tarefa assíncrona. Inicialmente, o código apenas multiplica uma parte da matriz utilizando só 2 threads. Logo, nossa tarefa para essa seção é ultilizar 5 threads em paralelo, desta forma, multiplicando o resto dos elementos da matriz, por coeficientes diferentes. Nesta sessão é importante manter o paradigma da memória compartilhada em mente.

# Resultados e Discussões

## Seção 1 - Multiplicação de matrizes

Começamos o processo de paralelização do código definindo os laços que executam a multiplicação entre as matrizes dentro de uma região paralela, logo depois, adicionamos mais uma técnica de profilling dentro do código capturando os tempos de início e término da multiplicação dentro das váriaveis t1 e t2:

In [ ]:
t1 = omp_get_wtime(); //Tempo quando loops iniciam
  
//Multiplicação de matrizes
#pragma omp parallel for private(i, j, k) // Comando para paralelização dos loops 
for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
        for(k = 0; k < size; k++)
            C[i * size + j] += A[i * size + k] * B[k * size + j];
            
t2 = omp_get_wtime(); //Tempo quando os loops acabam

Dentro dessa região, o código será paralelizado automaticamente privando as variáveis i, j e k, além disso também temos acesso ao tempo deste processo, diminuindo t1-t2 e imprimindo no terminal. No final do processo de paralelização teremos um código parecido com esse:

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void initializeMatrix(int *matrix, int size) //Preenche a matrix com numeros aleatorios
{
  for (int i = 0; i < size; i++)
    for (int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size) //Percorre a Matrix, printando ela
{
  for (int i = 0; i < size; i++)
  {
    for (int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main (int argc, char **argv)
{
 
  //Define as variáveis
  int size = atoi(argv[1]);  
  int i, j, k;
  double t1, t2;

  //Aloca as matrizes
  int  *A = (int *) malloc (sizeof(int)*size*size);
  int  *B = (int *) malloc (sizeof(int)*size*size);
  int  *C = (int *) malloc (sizeof(int)*size*size);
 
  //Preenche matrizes com numeros aleatorios
  initializeMatrix(A, size);
  initializeMatrix(B, size);

  t1 = omp_get_wtime(); //Tempo quando os loops iniciam
  
  //Multiplicação de matrizes
  #pragma omp parallel for private(i, j, k) // Comando para paralelização de loops 
    for(i = 0; i < size; i++)
      for(j = 0; j < size; j++)
        for(k = 0; k < size; k++)
          C[i * size + j] += A[i * size + k] * B[k * size + j];
 
  t2 = omp_get_wtime(); //Tempo quando os loops acabam
 
  //Printa matrizes
  //printMatrix(A,size);
  //printMatrix(B,size);
  //printMatrix(C,size);
 
  //Printa tamanho da matriz e tempo para multiplicação
  printf("%d\t%f\n",size,t2-t1);

  return 0;

}



1000	0.000000


Como proposto na prática, para confirmar e homologar a efetividade da paralelização de um código, precisamos comparar o código paralelizado ao original, não paralelizado. Para isso, o grupo usa um script para automatizar o processo, executando o código várias vezes, variando o número de threads e tamanho das matrizes, desta forma, coletando os dados e construindo os gráficos relacionais entre speedup do problema e número de threads e número de threads com velocidade de execução. Para executar o script basta executar:

In [ ]:
!bash START.sh

### Gráfico relacional entre tamanho da matriz e speedup do problema:

![Figure 1](https://user-images.githubusercontent.com/80331486/187289071-c89579df-b985-4dcb-849a-f3150c41aa9a.png)

No primeiro gráfico, podemos observar a relação entre speedup e tamanho das matrizes, em geral, 4 threads se mostra com uma taxa de speedup melhor, ficando para trás em poucos casos.

### Gráfico relacional entre tempo de execução e tamanho das matrizes:

![Figure 2](https://user-images.githubusercontent.com/80331486/187289256-d6a171c4-616f-47fd-b2cf-310f6295e9dd.png)

No segundo gráfico, podemos ver o tempo de execução caindo, conforme o número de threads aumenta, em paralelo, o tempo de execução subindo, conforme o tamanho das matrizes cresce. Segundo a teoria$^{[2]}$, existe um número ideal de threads tal que aumenta esse número representaria um aumento no tempo de execução, no entanto, numa máquina com poucas threads é díficil estimar este número ideal, mas segundo o gráfico, 4 threads é o número com maior eficiêcia.

## Seção 2 - Tarefas Assíncronas.

Executando o código fornecido inicialmente pela prática com:

In [ ]:
!gcc asyncTaskOpenMP.c -o asyncTaskOpenMP -fopenmp

In [ ]:
!./asyncTaskOpenMP 10 2

Ele imprime uma matriz 10x10 preenchida cincos, mais abaixo, ele imprime novamente a mesma matriz, agora com as quatro primeiras colunas mutiplicadas por 10 e 20. Estudando o funcionamento do código percebemos que dentro da área de paralelização, o código pega o id da threads que está executando o chunck e baseado nisso, determina através de condicionais para ditar oquê cada thread deve ou não fazer:

In [ ]:
//Criando área de paralelização, privando row e column
#pragma omp parallel private(row, column)
  {
    int id = omp_get_thread_num();

    if(id == 0) // Condicional para a thread de id de número 0
    {
      //Multiplicação das colunas por k1
      for(row = 0; row < n; row++)
        for(column = 0; column < block_size; column++)
          matrix[row][column] *= k1;
    }

    if(id == 1) // Condicional para a thread de id de número 1
    {
      //Multiplicação das colunas por k2
      for(row = 0; row < n; row++)
        for(column = block_size; column < block_size*2; column++)
          matrix[row][column] *= k2;
    }
}

Baseado nisso, o grupo usa a mesma tática, mas agora para utilizar 3 outras threads na operação com outros coeficientes em outras colunas. No final, obtemos o seguinte código:  

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#define SIZE_MATRIX 10

int main(int argc, char **argv)
{
  //Definido as váriaveis
  int n = atoi(argv[1]);
  int block_size = atoi(argv[2]);
  int matrix[n][n], k1 = 10, k2 = 20, k3 = 30,k4 = 40,k5 = 50;
  int i, j, row, column;

  //Preenchendo e imprimindo a matriz
  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
    {
      matrix[i][j] = 5;
      printf("%d\t", matrix[i][j]);
    }
    printf("\n");
  }

  printf("\n\n");

  //Definindo o número de threds
  omp_set_num_threads(5);

  //Criando área de paralelização, privando row e column
  #pragma omp parallel private(row, column)
  {
    int id = omp_get_thread_num();

    if(id == 0) // Condicional para a thread de id de número 0
    {
      //Multiplicação das colunas por k1
      for(row = 0; row < n; row++)
        for(column = 0; column < block_size; column++)
          matrix[row][column] *= k1;
    }else{
      if(id == 1){// Condicional para a thread de id de número 1
        //Multiplicação das colunas por k2
        for(row = 0; row < n; row++)
          for(column = block_size; column < block_size*2; column++)
            matrix[row][column] *= k2;
      }else{
        if(id == 2){ // Condicional para a thread de id de número 2
          //Multiplicação das colunas por k3
          for(row = 0; row < n; row++)
            for(column = block_size*2; column < block_size*3; column++)
              matrix[row][column] *= k3;
        }else{
          if(id == 3){ // Condicional para a thread de id de número 3
            //Multiplicação das colunas por k4
            for(row = 0; row < n; row++)
              for(column = block_size*3; column < block_size*4; column++)
                matrix[row][column] *= k4;
          }else{
            if(id == 4){ // Condicional para a thread de id de número 4
              //Multiplicação das colunas por k5
              for(row = 0; row < n; row++)
                for(column = block_size*4; column < block_size*5; column++)
                  matrix[row][column] *= k5;
            }
          }
        }
      }
    }
  }

  //Imprimindo matriz resultante
  for(i = 0; i < n; i++)
  {
    for(j = 0; j < n; j++)
      printf("%d\t", matrix[i][j]);
    printf("\n");
  }

  return 0;
}

Compilando e executando novamente esse código, obtemos uma matrizes com todas as colunas com multiplos de 5: 10, 20,30,40,50, onde cada threads executou uma dupla de colunas diferente, ao mesmo tempo.  

# Conclusões

Para resumir, durante a execução das práticas o grupo desenvolveu habilidades relacionadas ao uso da biblioteca OPENMP em problemas simples de otimização de código, criação de scripts em shellscript e análise de dados. Atráves da primeira sessão, observa-se na prática que o tempo de execução caí de acordo com o aumento do número de threads utilizadas no processo, enquanto na segunda sessão experimentamos um maior domíniom das threads repartindo manualmente as tarefas de cada uma. Portando, a prática HANDS-ON-1 foi um pontapé inicial para os estudos de programação paralela, estabelecendo conceitos básicos e iniciando o processo de otimização de problemas.

# Reconhecimentos
Todo o conceito da prática e orientação para o desenvolvimento da atividade foi realizada pelo professor Murillo Boratto, pesquisador do centro de supercomputação SENAI CIMATEC.

# Referências

[1] M. Boratto. Hands-On Supercomputing with Parallel Computing. Available: https://github.com/
muriloboratto/Hands-On-Supercomputing-with-Parallel-Computing. 2022.

[2] B. Chapman, G. Jost and R. Pas. Using OpenMP: Portable Shared Memory Parallel Programming. The
MIT Press, 2007, USA.